# KVecinos en un dataset de detección de spam

Se propone emplear un clasificador basado en distancias sobre el dataset id=44 de openml de detección de Spam. Son un total de 4601 muestras con 57 características.

In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

## Descarga del dataset Spam
X, y = fetch_openml(data_id=44, as_frame=False, cache=True, return_X_y=True)
print(X.shape)

## Partición train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)


(4601, 57)


## El clasificador por los vecinos más cercanos

In [2]:
from sklearn.neighbors import KNeighborsClassifier

kv = KNeighborsClassifier()
acc=kv.fit(X_train,y_train).score(X_test,y_test)

print(f'Precisión: {acc:.1%}')

Precisión: 79.5%


**Ejercicio:** Explora el principal parámetros del KNN (n_neighbors) y realiza una búsqueda mediante alguna técnica de optimización ya vista en la práctica anterior

In [3]:
# Solución

from sklearn.model_selection import GridSearchCV

G = {"n_neighbors":[1,3,4,5,10]}

GS = GridSearchCV(KNeighborsClassifier(), G, scoring='accuracy', refit=True, cv=5)

acc=GS.fit(X_train,y_train).score(X_test,y_test)
print(f'Precisión: {acc:.1%}')

Precisión: 82.1%


## Mejoras

La función de distancia empleada por defecto es la distancia euclídea. Dicha distancia requiere un preproceso de las muestras para que tengan una escala similar todas ellas. Además KNN podría beneficiarse de una proyección mediante PCA con el fin de reducir la dimensionalidad.

**Ejercicio:** Implementa un pipeline con la normalización (ESO SIGNIFICA USAR SCALER) de los datos y un PCA, seguido del KNN. Busca los mejores parámetros. Se podría conseguir una tasa de acierto >90%.


In [4]:
import warnings; warnings.filterwarnings('ignore')
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV;
from sklearn.decomposition import PCA;
from sklearn.pipeline import Pipeline;

X, y = fetch_openml(data_id=44, as_frame=False, cache=True, return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)
pca = PCA(); kn = KNeighborsClassifier(); scaler = StandardScaler();
pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("kn", kn)])

G = {"pca__n_components": [4,8,16,32,64,128], "kn__n_neighbors":[2,3,4,5,6,7,8,9,10,24]}

GS = GridSearchCV(pipe, G, scoring='accuracy', refit=True, cv=5)

acc=GS.fit(X_train,y_train).score(X_test,y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

Precisión: 91.1% con {'kn__n_neighbors': 7, 'pca__n_components': 16}


También podríamos probar diferentes funciones de distancia [sklearn distances](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics) a emplear en el parámetro "metric". Así mismo podríamos explorar el parámetro "weights" que pondera el voto de cada vecino de forma diferente según el parámetro escogido.

**Ejercicio:** prueba también diferentes métricas y "weights" junto con todo lo anterior. Emplea el BayessianOpt visto en la práctica anterior.

In [8]:
import warnings; warnings.filterwarnings('ignore')
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV;
from sklearn.decomposition import PCA;
from sklearn.pipeline import Pipeline;
from skopt import BayesSearchCV;

X, y = fetch_openml(data_id=44, as_frame=False, cache=True, return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)
pca = PCA(); kn = KNeighborsClassifier(); scaler = StandardScaler();
pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("kn", kn)])

G = {"pca__n_components": [4,8,16,32,50], "kn__n_neighbors":[2,3,4,5,6,7,8,9,10], "kn__weights":['uniform', 'distance'], "kn__metric":['euclidean', 'manhattan']}
GS = BayesSearchCV(pipe, G, scoring='accuracy', refit=True, cv=5)

acc=GS.fit(X_train,y_train).score(X_test,y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

Precisión: 92.3% con OrderedDict([('kn__metric', 'manhattan'), ('kn__n_neighbors', 7), ('kn__weights', 'distance'), ('pca__n_components', 16)])


## Olivetti Faces

Prueba ahora el clasificador KNN junto con todos los parámetros y preprocesos que creas convenientes sobre el dataset de reconocimiento facial de Olivetti.

In [10]:
import warnings; warnings.filterwarnings('ignore')
from sklearn.datasets import fetch_olivetti_faces
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV;
from sklearn.decomposition import PCA;
from sklearn.pipeline import Pipeline;
from skopt import BayesSearchCV;

X, y = fetch_olivetti_faces(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)
pca = PCA(); kn = KNeighborsClassifier(); scaler = StandardScaler();
pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("kn", kn)])

G = {"pca__n_components": [4,8,16,32,50], "kn__n_neighbors":[2,3,4,5,6,7,8,9,10], "kn__weights":['uniform', 'distance'], "kn__metric":['euclidean', 'manhattan']}
GS = BayesSearchCV(pipe, G, scoring='accuracy', refit=True, cv=5)

acc=GS.fit(X_train,y_train).score(X_test,y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to /home/yassin/scikit_learn_data
Precisión: 93.8% con OrderedDict([('kn__metric', 'manhattan'), ('kn__n_neighbors', 2), ('kn__weights', 'distance'), ('pca__n_components', 50)])
